In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats 
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.multicomp import pairwise_tukeyhsd
import statsmodels.formula.api as smf


In [2]:
data=pd.read_csv('insurance.csv')

In [3]:
data.drop('id',axis=1,inplace = True)

In [4]:
data.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


In [5]:
numerical=data.select_dtypes(include=np.number)
print(len(numerical.columns))
numerical.columns

4


Index(['age', 'bmi', 'children', 'charges'], dtype='object')

In [6]:
categorical=data.select_dtypes(include=object)
print(len(categorical.columns))
categorical.columns

3


Index(['sex', 'smoker', 'region'], dtype='object')

In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   int64  
 1   sex       1338 non-null   object 
 2   bmi       1338 non-null   float64
 3   children  1338 non-null   int64  
 4   smoker    1338 non-null   object 
 5   region    1338 non-null   object 
 6   charges   1338 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 73.3+ KB


In [8]:
data.isnull().sum()

age         0
sex         0
bmi         0
children    0
smoker      0
region      0
charges     0
dtype: int64

In [9]:
data['chrges']=np.log(data['charges'])

In [10]:
dummy_enc=pd.get_dummies(categorical,drop_first=True)

In [11]:
data1=pd.concat([dummy_enc,numerical],axis=1)

In [12]:
data1.head()

,sex_male,smoker_yes,region_northwest,region_southeast,region_southwest,age,bmi,children,charges
0,0,1,0,0,1,19,27.900,0,16884.92400
1,1,0,0,1,0,18,33.770,1,1725.55230
2,1,0,0,1,0,28,33.000,3,4449.46200
3,1,0,1,0,0,33,22.705,0,21984.47061
4,1,0,1,0,0,32,28.880,0,3866.85520


In [13]:
#Library for VIF
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [14]:
def calculate_vif(dataset):
    vif = pd.DataFrame()
    vif["features"] = dataset.columns
    vif["VIF_Values"] = [variance_inflation_factor(dataset.values, i) for i in range(dataset.shape[1])]
    return(vif)

**1. take dataset from input**

**2. create a empty dataframe**

**3. one columns features means name of the column dataset**

**second  vif_value means what is the vif value for particular column**

# correlation  test to remove highly impact

In [15]:
data1.corr()

,sex_male,smoker_yes,region_northwest,region_southeast,region_southwest,age,bmi,children,charges
sex_male,1.000000,0.076185,-0.011156,0.017117,-0.004184,-0.020856,0.046371,0.017163,0.057292
smoker_yes,0.076185,1.000000,-0.036945,0.068498,-0.036945,-0.025019,0.003750,0.007673,0.787251
region_northwest,-0.011156,-0.036945,1.000000,-0.346265,-0.320829,-0.000407,-0.135996,0.024806,-0.039905
region_southeast,0.017117,0.068498,-0.346265,1.000000,-0.346265,-0.011642,0.270025,-0.023066,0.073982
region_southwest,-0.004184,-0.036945,-0.320829,-0.346265,1.000000,0.010016,-0.006205,0.021914,-0.043210
age,-0.020856,-0.025019,-0.000407,-0.011642,0.010016,1.000000,0.109272,0.042469,0.299008
bmi,0.046371,0.003750,-0.135996,0.270025,-0.006205,0.109272,1.000000,0.012759,0.198341
children,0.017163,0.007673,0.024806,-0.023066,0.021914,0.042469,0.012759,1.000000,0.067998
charges,0.057292,0.787251,-0.039905,0.073982,-0.043210,0.299008,0.198341,0.067998,1.000000


In [16]:
features = data1.iloc[:,:-1]
calculate_vif(features)

,features,VIF_Values
0,sex_male,2.003185
1,smoker_yes,1.261233
2,region_northwest,1.890281
3,region_southeast,2.265564
4,region_southwest,1.960745
5,age,7.686965
6,bmi,11.358443
7,children,1.809930


From the above VIF observation, only one variable(i.e., Bmi) is more than 10, hence we can drop that variable.

In [17]:
x=data1.drop('charges',axis=1)
y=data1['charges']

In [18]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.30, random_state=25)

In [19]:
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(936, 8)
(402, 8)
(936,)
(402,)


In [20]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.linear_model import LinearRegression
from sklearn import metrics
import statsmodels.api as sm


In [21]:
model=sm.OLS(y_train,x_train).fit()
print(model.summary())

                                 OLS Regression Results                                
Dep. Variable:                charges   R-squared (uncentered):                   0.879
Model:                            OLS   Adj. R-squared (uncentered):              0.878
Method:                 Least Squares   F-statistic:                              840.2
Date:                Wed, 22 Sep 2021   Prob (F-statistic):                        0.00
Time:                        20:39:56   Log-Likelihood:                         -9505.3
No. Observations:                 936   AIC:                                  1.903e+04
Df Residuals:                     928   BIC:                                  1.907e+04
Df Model:                           8                                                  
Covariance Type:            nonrobust                                                  
                       coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------

In [22]:
l_reg = LinearRegression()
l_reg.fit(x_train,y_train)
l_acc = r2_score(y_test,l_reg.predict(x_test))


**i want to add predicted values in the data (train and test both), how to do**

In [23]:
y_pred = l_reg.predict(x_test)

In [24]:
y_pred=pd.Series(y_pred)

In [25]:
pd.concat([y_test,y_pred],axis=1)

,charges,0
0,NaN,11703.342652
1,NaN,7346.629674
2,NaN,13933.829293
3,NaN,31647.225510
4,NaN,32520.420053
...,...,...
1322,12981.3457,NaN
1326,7050.0213,NaN
1327,9377.9047,NaN
1329,10325.2060,NaN


In [26]:
(l_reg.score(x_train,y_train))

0.757956680840197

In [27]:
(l_reg.score(x_test,y_test))

0.7299796870282933

# Durbin-Watson Test

In [28]:
model1=smf.ols(formula='charges ~ age + sex + bmi + children + smoker + region',data=data).fit()
print(model1.summary())

                            OLS Regression Results                            
Dep. Variable:                charges   R-squared:                       0.751
Model:                            OLS   Adj. R-squared:                  0.749
Method:                 Least Squares   F-statistic:                     500.8
Date:                Wed, 22 Sep 2021   Prob (F-statistic):               0.00
Time:                        20:39:56   Log-Likelihood:                -13548.
No. Observations:                1338   AIC:                         2.711e+04
Df Residuals:                    1329   BIC:                         2.716e+04
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept           -1.194e+04    

In [29]:
from statsmodels.stats.stattools import durbin_watson

In [30]:
#perform Durbin-Watson test
durbin_watson(model1.resid)

2.088422998667309

# The test statistic is 2.088. Since this is within the range of 1.5 and 2.5, we would consider autocorrelation not to be problematic in this regression model.

#shapiro wilk test for NORMALITY OF resisulas of train and test data

In [31]:
pred_1 = model.predict(x_train)

In [32]:
pred_2=model.predict(x_test)

In [33]:
res_train=y_train - pred_1

In [34]:
res_test=y_test - pred_2

In [35]:
from scipy.stats import shapiro

In [44]:

stat, p = shapiro(res_train)
print('Statistics=%.3f, p=%.3f' % (stat, p))
# interpret
alpha = 0.05
if p > alpha:
    print('The Null Hypothesis can be reject H0)')
else:
    print('She Null Hypothesis can not be rejectt H0)')

Statistics=0.841, p=0.000
She Null Hypothesis can not be rejectt H0)


In [43]:
stat, p = shapiro(res_test)
print('Statistics=%.3f, p=%.3f' % (stat, p))
# interpret
if p > alpha:
    print('The Null Hypothesis can be reject H0)')
else:
    print('She Null Hypothesis can not be rejectt H0)')

Statistics=0.868, p=0.000
She Null Hypothesis can not be rejectt H0)


#kolmogorov-smirnov test for NORMALITY OF resisulas of train and test data

# The Kolmogorov–Smirnov statistic quantifies a distance between the empirical distribution function of the sample and the cumulative distribution function of the reference distribution, or between the empirical distribution functions of two samples.

In [38]:
from scipy.stats import kstest

In [41]:
test_stat = kstest(res_train, 'norm')
print('Statistics=%.3f, p=%.3f' % (test_stat))
# interpret
alpha = 0.05
if p > alpha:
    print('two distributions are identical (fail to reject H0)')
else:
    print('two distributions are not identical  (reject H0)')

Statistics=0.745, p=0.000
two distributions are not identical  (reject H0)


In [42]:
test_stat = kstest(res_test, 'norm')
print('Statistics=%.3f, p=%.3f' % (test_stat))
# interpret
alpha = 0.05
if p > alpha:
    print('two distributions are identical  (fail to reject H0)')
else:
    print('two distributions are not identical  (reject H0)')

Statistics=0.754, p=0.000
two distributions are not identical  (reject H0)
